In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from helpers.utils import creat_datasets, creating_val_data, reset_graph, grid_serach, read_pickle_file, write_pickle_file
from helpers.utils import creating_train_val_test_datasets, tensorboard_callback,  save_best_model_callback
from helpers.bln_layer import  bln_layer
from helpers.dense_layer import  dense_layer
from helpers.bln_callback import bln_callback 




from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [2]:
tf.config.experimental_run_functions_eagerly(True)

In [3]:
tf.__version__

'2.1.0'

### Loading data

In [4]:
random_seed = 100
minibatch = 25
buffersize = 60000
number_valid_sampels = 5000 # number of validation data
epochs = 10
learning_rate = 0.003

#((25000  - 5000 )/ 25 ) * 1
number_batches_train = 800 # number of batches to train, each batch of size minibatch parameter
number_batches_valid = 200 # number of batches to validate, each batch of size minibatch parameter (( 5000 )/ 25 ) * 1
num_classes = 1

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)


In [5]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...


C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [6]:
x_train, y_train, x_valid, y_valid = creating_val_data(x_train, y_train,
                                                       number_valid_sampels = number_valid_sampels,
                                                       random_seed=random_seed)

train_dataset, valid_dataset, test_dataset = creating_train_val_test_datasets(x_train, y_train,
                                                                              x_test, y_test,
                                                                              x_valid, y_valid, 
                                                                              minibatch = minibatch,
                                                                              buffersize= buffersize,
                                                                              random_seed=random_seed)

In [7]:
train_dataset

<PrefetchDataset shapes: ((25, 80), (25,)), types: (tf.int32, tf.int64)>

# 1. Using Batch Layer Normalization Layer

In [8]:
def BLNLayer_model(inputshape= (80), max_features = 20000, embed_size = 512, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1, batch_size = 10,
                          b_mm = True, b_mv = True, f_mm = False, f_mv = False):
    
   
    # building the model
  
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn1', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn2', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn3', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x)
           
    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])

In [9]:
model_bln_layer = BLNLayer_model(inputshape= (80), max_features = max_features, 
                                        embed_size = 512, random_seed = random_seed,
                                        lstm_unit = 128 , dense_units= num_classes,
                                        batch_size = minibatch,
                                        b_mm = False, b_mv = False,
                                        f_mm = False, f_mv = False
                                        )



In [10]:
model_bln_layer.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 512)             10240000  
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             328192    
_________________________________________________________________
bn1 (bln_layer)              (25, 80, 128)             24738     
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
bn2 (bln_layer)              (25, 64)                  308       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

# callbacks

In [11]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bln_layer_imdb_wohletraining_batchSize_' + str(minibatch) 
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

# Callback for resetting moving mean and variances at the end of each epoch
bln_layer_cb = bln_callback()

bln_layer_cb_list = [save_bm_cb, tb_cb, bln_layer_cb]

In [12]:
model_bln_layer.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                               loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                               metrics = [tf.keras.metrics.BinaryAccuracy()])

In [13]:
model_bln_layer_history =  model_bln_layer.fit(train_dataset.take(number_batches_train), epochs=epochs,
                                                verbose=1, callbacks=bln_layer_cb_list,
                                                validation_data=valid_dataset.take(number_batches_valid),
                                                shuffle=True)

Train for 800 steps, validate for 200 steps
Epoch 1/10
800/800 [==============================] - 703s 878ms/step - loss: 0.4781 - binary_accuracy: 0.7599 - val_loss: 0.3920 - val_binary_accuracy: 0.8086
Epoch 2/10
800/800 [==============================] - 737s 921ms/step - loss: 0.2744 - binary_accuracy: 0.8839 - val_loss: 0.4326 - val_binary_accuracy: 0.8208
Epoch 3/10
800/800 [==============================] - 748s 935ms/step - loss: 0.1499 - binary_accuracy: 0.9447 - val_loss: 0.5785 - val_binary_accuracy: 0.8186
Epoch 4/10
800/800 [==============================] - 744s 929ms/step - loss: 0.0933 - binary_accuracy: 0.9675 - val_loss: 0.6821 - val_binary_accuracy: 0.8092
Epoch 5/10
800/800 [==============================] - 712s 890ms/step - loss: 0.0658 - binary_accuracy: 0.9757 - val_loss: 0.8261 - val_binary_accuracy: 0.8178
Epoch 6/10
800/800 [==============================] - 751s 938ms/step - loss: 0.0464 - binary_accuracy: 0.9821 - val_loss: 0.9209 - val_binary_accuracy: 0.8

In [14]:
del model_bln_layer, save_bm_cb, tb_cb, bln_layer_cb, bln_layer_cb_list
reset_graph()

session is clear


In [15]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bln_layer_imdb_wohletraining_batchSize_25/best_model.h5'

In [16]:
save_eval_path = "./logs/" + folder_name + '/'+ str(number_batches_train) +"_sorted_evaluation.pkl"
evaluation = grid_serach(BLNLayer_model, test_dataset,
                         batch_size = minibatch, sort=True,
                         save_eval_path = save_eval_path,
                         weights_path = weights_path,
                         loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                         metrics = tf.keras.metrics.BinaryAccuracy())



1000/1000 [==============================] - 308s 308ms/step - loss: 0.4339 - binary_accuracy: 0.7815
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.4338963188454509, 0.78148]}
session is clear
1000/1000 [==============================] - 301s 301ms/step - loss: 0.4335 - binary_accuracy: 0.7832
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.4338963188454509, 0.78148], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.4335413608327508, 0.7832]}
session is clear
1000/1000 [==============================] - 296s 296ms/step - loss: 0.4336 - binary_accuracy: 0.7816
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.4338963188454509, 0.78148], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.4335413608327508, 0.7832], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.43361701464653013, 0.78156]}
session is clear
1000/1000 [==============================] - 298s 298ms/step - loss: 0.4333 - binary_accuracy: 0.7835
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.4338963188454509, 0.78148], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.43

1000/1000 [==============================] - 296s 296ms/step - loss: 0.3831 - binary_accuracy: 0.8179
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.4338963188454509, 0.78148], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.4335413608327508, 0.7832], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.43361701464653013, 0.78156], 'Bmm_True Bmv_True Fmm_False Fmv_False': [0.4332664158344269, 0.78348], 'Bmm_True Bmv_False Fmm_True Fmv_True': [0.6696520148664713, 0.65568], 'Bmm_True Bmv_False Fmm_True Fmv_False': [0.6758395366966724, 0.65512], 'Bmm_True Bmv_False Fmm_False Fmv_True': [0.6689283060282469, 0.65608], 'Bmm_True Bmv_False Fmm_False Fmv_False': [0.6750893536508084, 0.6554], 'Bmm_False Bmv_True Fmm_True Fmv_True': [0.3821256238147616, 0.82472], 'Bmm_False Bmv_True Fmm_True Fmv_False': [0.38203894811868666, 0.82496], 'Bmm_False Bmv_True Fmm_False Fmv_True': [0.3821210457459092, 0.82468], 'Bmm_False Bmv_True Fmm_False Fmv_False': [0.38203524985164405, 0.82492], 'Bmm_False Bmv_False Fmm_True Fmv_Tr

In [19]:
evaluation

[('Bmm_False Bmv_True Fmm_False Fmv_False', [0.38203524985164405, 0.82492]),
 ('Bmm_False Bmv_True Fmm_True Fmv_False', [0.38203894811868666, 0.82496]),
 ('Bmm_False Bmv_True Fmm_False Fmv_True', [0.3821210457459092, 0.82468]),
 ('Bmm_False Bmv_True Fmm_True Fmv_True', [0.3821256238147616, 0.82472]),
 ('Bmm_False Bmv_False Fmm_False Fmv_True', [0.38306253392994405, 0.81788]),
 ('Bmm_False Bmv_False Fmm_True Fmv_True', [0.3830686711072922, 0.81796]),
 ('Bmm_False Bmv_False Fmm_False Fmv_False', [0.38332282204926016, 0.81816]),
 ('Bmm_False Bmv_False Fmm_True Fmv_False', [0.3833300884962082, 0.81824]),
 ('Bmm_True Bmv_True Fmm_False Fmv_False', [0.4332664158344269, 0.78348]),
 ('Bmm_True Bmv_True Fmm_True Fmv_False', [0.4335413608327508, 0.7832]),
 ('Bmm_True Bmv_True Fmm_False Fmv_True', [0.43361701464653013, 0.78156]),
 ('Bmm_True Bmv_True Fmm_True Fmv_True', [0.4338963188454509, 0.78148]),
 ('Bmm_True Bmv_False Fmm_False Fmv_True', [0.6689283060282469, 0.65608]),
 ('Bmm_True Bmv_False